In [96]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import boto3
from botocore.exceptions import ClientError
import os
import json

In [2]:
twitter_data = pd.read_csv("Corona_NLP_train.csv", encoding = "ISO-8859-1")

In [3]:
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


Для начала немного преподготовим наши данные, и объеденим Extremely Negative с Negative  
и Extremely Positive с Positive

In [4]:
for i in range(0, len(twitter_data)):
    if(twitter_data['Sentiment'][i]=='Extremely Negative'):
        twitter_data['Sentiment'][i]='Negative'
    elif(twitter_data['Sentiment'][i]=='Extremely Positive'):
        twitter_data['Sentiment'][i]='Positive'

<ipython-input-4-9eae80cfb354>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data['Sentiment'][i]='Negative'
<ipython-input-4-9eae80cfb354>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  twitter_data['Sentiment'][i]='Positive'


Теперь очистим данные от хэштегов, ссылок и прочего

In [41]:
def clean(data):
    out = []
    for i in range(0, len(data)):
        temp = data[i]
        temp = re.sub("@\S+", " ", temp)
        temp = re.sub("https*\S+", " ", temp)
        temp = re.sub("#\S+", " ", temp)
        temp = re.sub("\'\w+", '', temp)
        temp = re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
        temp = re.sub(r'\w*\d+\w*', '', temp)
        temp = re.sub('\s{2,}', " ", temp)
        list_of_temp = temp.split(" ")
        list_of_temp = list(map(lambda x: x.lower(), list_of_temp))
        temp = ' '.join(list_of_temp)
        out.append(temp)
    return out

In [42]:
out = clean(list(twitter_data['OriginalTweet']))

In [43]:
twitter_data.drop(['OriginalTweet'], axis=1)

,UserName,ScreenName,Location,TweetAt,Sentiment
0,3799,48751,London,16-03-2020,Neutral
1,3800,48752,UK,16-03-2020,Positive
2,3801,48753,Vagabonds,16-03-2020,Positive
3,3802,48754,NaN,16-03-2020,Positive
4,3803,48755,NaN,16-03-2020,Negative
...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Neutral
41153,44952,89904,NaN,14-04-2020,Negative
41154,44953,89905,NaN,14-04-2020,Positive
41155,44954,89906,NaN,14-04-2020,Neutral


In [44]:
twitter_data['OriginalTweet'] = out

In [45]:
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,and and,Neutral
1,3800,48752,UK,16-03-2020,advice talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths to give elder...,Positive
3,3802,48754,NaN,16-03-2020,my food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,me ready to go at supermarket during the outbr...,Negative


Теперь поделим наш датасет на классы

In [14]:
list_of_labels = list(twitter_data['Sentiment'].unique())
list_of_labels

['Neutral', 'Positive', 'Negative']

In [94]:
def get_twitts(label):
    l = twitter_data[twitter_data.Sentiment == label]
    l = list(l['OriginalTweet'])
    
    f = open(f"{label}.txt", "w")
    for twitt in l[0:int(len(l)/10)]:
        f.write(twitt + "\n")
    f.close()

In [95]:
get_twitts('Positive')
get_twitts('Negative')
get_twitts('Neutral')

In [113]:
class ComprehendDetect:
    def __init__(self, comprehend_client):
        self.comprehend_client = comprehend_client

    def detect_key_phrases(self, text, language_code):
        try:
            response = self.comprehend_client.detect_key_phrases(
                Text=text, LanguageCode=language_code)
            phrases = response['KeyPhrases']
        except ClientError:
            raise
        else:
            return phrases

    def detect_sentiment(self, text, language_code):
        try:
            response = self.comprehend_client.detect_sentiment(
                Text=text, LanguageCode=language_code)
        except ClientError:
            raise
        else:
            return response
        
access_key_id = os.getenv('MY_ACCESS_KEY_ID')
secret_access_key = os.getenv('MY_SECRET_ACCESS_KEY')
comp = boto3.client('comprehend', region_name="us-east-1", aws_session_token = "IQoJb3JpZ2luX2VjEN3//////////wEaCXVzLXdlc3QtMiJIMEYCIQDwDaa3iRyHxfv1/17JkOINlWpmcYr8v0T89LUbq+af+gIhALkPo+Zjl6fiPWArPkfYHs3la8SqhhsKRZpC7X4eMacgKrQCCGYQABoMNjEzMjg5NjM4NDE4Igxk84kQHvp/EPLx0YQqkQKEgMceKtfS+4ir5IrcKS5P1OHWkacRYULVcjJ5jiHI0cBdzc1IiCPoTtHrA2tvHPjbevT5d5SAwDqCKSoruYradaJJ9jQa94BkfUGaoXlGLDJETg/wvS9d4Ae+vyZiMN/+FjmB7DoTRlWbxk/Z/+bnWJrUXptHv2xCNPqowUsGyPdSbe2NYj1tpBXjk7SPu1przhlOinnMaQQWPrMKLmhFrhrisQLi0rUPLsUOkK6hEfRATEuO0yLPN29DCzpqhheRNINxwP99yv3qV3dLsREQj3umsuH65iLV9W1ECM3tGNgGXsfaGKbMGSM89wk3OFCWa0KP/la66TYSHmuMg/M3d7J8p+cohTIvfa9AbkgnYUQwk+zbhAY6nAEaU9q43L2UGV0hQrDqmk11anPqSK5QH1G9KdsVpGlLPmnvrxIeGIVfeRV3qrdxrZyjOlrk+B9/mymhNSr5z6uM8QNhfC5spHQXb67wUVmjdHWFf55FpcwDuH+a4xsVHBSutN9k4kP2gGeD4ttgCoAA7XdOqaIbb7Yjk5iN0r48G1lUEPue4vvpWgHE/ax3mW7IW0UwgdR0hEPbIsc=")

comp_detect = ComprehendDetect(comp)

phrases = comp_detect.detect_key_phrases('', 'en')

In [111]:
jsons = []
with open("sentiment_from_comprehend.txt") as f:
    for line in f:
        jsons.append(json.loads(line))
        
score = 0
for j in jsons:
    original_sentiment = j['File'].split(".")[0].lower()
    pred_sentiment = j['Sentiment'].lower()
    if original_sentiment == pred_sentiment:
        score += 1

Как мы видим с помощью амазон сервиса мы получили крайне низкий результат accuracy  
Лишь немного выше чем рандомный выбор с вероятностью угадать 1/3

In [114]:
accuracy = score / len(jsons) * 100
print(f"Accuracy = {accuracy}")

Accuracy = 39.499270782693245


In [115]:
key_phrases_dict = dict()